# Dipole angle tutorial

Here we analyse a box of water molecules exposed to an alternating electric field. 
To follow this tutorial, the data test files `electricfwater` of MAICoS are needed.

Simulation details - The system contains $\sim 5000$ water molecules simulated 
for 100 ps in the NVT ensemble at a temperature of 300 K. Periodic 
boundary conditions were employed in all directions and long range 
electrostatics were modelled using the PME method.
LINCS algorithm was used to constraint the H-Bonds. The alternating electric field
was applied along the $x$-axis in the form of a Gaussian laser pulse. 
Please check 
[gromacs electric field](https://manual.gromacs.org/2019-current/reference-manual/special/electric-fields.html)
for more details.

To follow this tutorial, the data test files airwater of MAICoS are needed. You can obtain them by cloning MAICoS repository:
```
    git clone git@gitlab.com:maicos-devel/maicos.git
```
The ``electricfwater`` data files are located in ``tests/data/electricfwater/``.

## Option 1: from the Python interpreter

First, let us ignore unnecessary warnings:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Then, import MAICoS, NumPy, MDAnalysis, and PyPlot:

In [ ]:
import maicos
import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

and set a few parameters for plotting purpose:

In [ ]:
fontsize = 25
font = {'family': 'sans', 'color':  'black', 
        'weight': 'normal', 'size': fontsize}
my_color_1 = np.array([0.090, 0.247, 0.560])
my_color_2 = np.array([0.235, 0.682, 0.639])
my_color_3 = np.array([1.000, 0.509, 0.333])
my_color_4 = np.array([0.588, 0.588, 0.588])

Define the path to the ``electricfwater`` data folder of MAICoS:

In [ ]:
datapath = "../../../../tests/data/electricfwater/"

Create a MDAnalysis universe, and extract its atoms:

In [ ]:
u = mda.Universe(datapath+"mdelectric.tpr",
                 datapath+"mdelectric.trr")
atoms = u.atoms

Important note: Because this calculation is computationaly heavy, 
let use isolate a small portion of the trajectory file. To perform the full 
calculation, just comment the following line:

In [ ]:
u.transfer_to_memory(stop = 100)

Let us print a few information about the trajectory file:

In [ ]:
print(f"The number of water molecules is {np.int32(u.atoms.n_atoms/3)}")
timestep = np.round(u.trajectory.dt,2)
print(f"The time interval between the frames is {timestep} ps")
total_time = np.round(u.trajectory.totaltime,2)
print(f"The total simulation time is {total_time} ps")

Run the MAICoS dipole angle function, selecting the $x$
axis using the `dim` keyword:

In [ ]:
dipangle = maicos.DipoleAngle(atoms, dim=0)
dipangle.run()

The run produces a python dictionary named ``dipangle.results``
with 4 keys linked to numpy arrays as values: 
1. timestep, 
2. cosine of dipole and x-axis, 
3. cosine squared, 
4. product of cosine of dipoles $i$ and $j$ with $i \ne j$.

Extract the time vector and the $\cos(\theta_i)$ data 
from the ``results`` attribute:

In [ ]:
t = dipangle.results["t"]
cos_theta_i = dipangle.results["cos_theta_i"]

 Plot the final results using :

In [ ]:
fig = plt.figure(figsize=(13,6.5))
ax1 = plt.subplot(1, 1, 1)
plt.xlabel(r"$t$ (ps)", fontdict=font)
plt.ylabel(r"cos($\theta_i$)", fontdict=font)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
ax1.plot(t, cos_theta_i, color=my_color_1, linewidth=4)
ax1.yaxis.offsetText.set_fontsize(20)
ax1.minorticks_on()
ax1.tick_params('both', length=10, width=2, which='major', direction='in')
ax1.tick_params('both', length=6, width=1.4, which='minor', direction='in')
ax1.xaxis.set_ticks_position('both')
ax1.yaxis.set_ticks_position('both')
ax1.spines["top"].set_linewidth(2)
ax1.spines["bottom"].set_linewidth(2)
ax1.spines["left"].set_linewidth(2)
ax1.spines["right"].set_linewidth(2)
ax1.yaxis.offsetText.set_fontsize(30)
minor_locator_y = AutoMinorLocator(2)
ax1.yaxis.set_minor_locator(minor_locator_y)
minor_locator_x = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator_x)
ax1.tick_params(axis='x', pad=10)
plt.show()

## Option 2: from the command line interface

Go to the ``electricfwater`` data folder of MAICoS, then use the ``maicos``
command directly from the terminal:
```
    cd tests/data/electricfwater/
    maicos DipoleAngle -s md.tpr -f md.trr -d 0
```

The output file ``dipangle.dat`` is similar to ``dipangle.results`` and contains the data in columns. To know the full
list of options, have a look at the ``Inputs``.